<a href="https://colab.research.google.com/github/teddy-teem/deep-learning/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk


In [3]:
document = """About Data Science Mentorship Program

What is the course fee for  Data Science Mentorship Program (DSMP 2.0)
The total course fee for the DSMP course is Rs 10,600. This includes the fee for both DSMP 1.0 and DSMP 2.0

What is the total duration of the course?
The total duration of the course is about 6-8 months.

Are Deep Learning and NLP a part of the DSMP 2.0  course?
No, NLP and Deep Learning both are not a part of this program’s curriculum.

What if I miss a live session? Will I get a recording of the session?
Yes, all our sessions are recorded, so even if you miss a session you can login into our portal and watch the recording as per your convenience.

Where can I find the class schedule?
You will find the class schedule in your course dashboard once you enroll for the course.

What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.


What is the language spoken by the instructor during the sessions?
Hinglish.

How will I be informed about the upcoming class?
You will get mail from our side before every session once you enroll in the course.

Can I do this course if I am from a non-tech background?
Yes, absolutely. However, you’ll need to be consistent.

I am late, can I join the program in the middle?
Absolutely, you can join the program any time.

If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you’re enrolled in the course, you will be able to see all the past content in your dashboard.

Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.

Will we do case studies in the program?
Yes, there are 5 end to end case studies and multiple smaller projects.

Do we offer any student discounts or promo codes?
We do offer discounts on our courses a few times a year, and we announce them through our social media channels.

On how many devices can I watch the videos simultaneously?
You can watch the videos on one device at a time.



Do you provide notes for the lectures?
We do not provide notes for our lectures. We encourage students to make their own notes as it aids in study retention.

What is the difference between being a paid member on your YouTube channel and your website?
While the content on our YouTube channel and website is similar, enrolling through our website offers additional benefits such as doubt clearance support and access to an exclusive community of learners on Discord and more.

Do I get access to your YouTube channel as a paid member after enrolling through your website?
No, the YouTube channel membership is separate from what we offer on our website.

Is there any upcoming batch for the Data Science Mentorship Program?
Currently, we do not have any plans to launch a new batch for DSMP. However, all previous lectures are available as recordings, allowing you to learn at your own pace.

Do you provide doubt clearance for the projects uploaded on CampusX YouTube channel?
We do not provide doubt clearance for any of our videos or projects uploaded on YouTube. However, you can ask your doubts in our Discord channel or in the comments section.






Payment/Registration related questions

Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/

What if I don’t like the course after making the payment? What is the refund policy?
You get a 7 days refund period from the day you have made the payment. To apply for refund, send us an email at support@campusx.in

I am living outside India and I am not able to make the payment on the website. What should I do?
You can contact us by sending a mail at support@campusx.in

I am from Pakistan and I am unable to make the payment?
We currently do not have the option to accept payment from Pakistan directly. If you’d still like to enroll for the course, you may ask a friend or family member living outside Pakistan to make the fee payment on your behalf.

Do you accept PayPal as a payment method?
Yes, we do. Please contact us at support@campusx.in for more details on using PayPal for payment.

I cannot pay the DSMP 2.0 fee as a one-time payment, is there any way to make a payment in installments?
At the moment, we do not have a monthly payment plan for the DSMP 2.0 course. However, if you have financial constraints, you can enroll in DSMP 1.0 by paying 799/- a month. Once you complete the course, you may apply for a discount coupon from the website.


Post registration queries

Till when can I view the paid videos on the website?
You can watch the videos till your subscription is valid, that is 3 years from the date of purchase for DSMP 2.0 and 2 years for DSMP 1.0

Why is lifetime validity not provided?
Because of the low course fee.

Do you assist with personal projects?
We do not provide support for personal projects due to time constraints and current commitments.

Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearing session.



If I join the program late, can I still ask past week doubts?
Yes, just select “past week's doubt” in the doubt clearance google form.

What is the validity of Doubt Support?
You will get doubt support for 1 year from the data of registration.



Certificate and Placement Assistance related queries

What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 10,600
You have to attempt all the course assessments.

I have read that Job Preparation is a part of this program. What comes under Job Preparation?
This is to clarify that Job Preparation does not mean Placement guarantee. We don't guarantee our learners any jobs or for that matter even interview calls. If you are planning to join this course just for placements, we’re afraid that this course might not be the best for you.

Here is what comes under Job Preparation
Portfolio Building sessions.
Interview Questions.
Sessions with industry mentors.
Discussion on Job hunting strategies.
"""

In [4]:
# tokenization

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
# tokeniz

tokens = word_tokenize(document.lower())

In [6]:
# build vocabulary

vocab = {'<unk>': 0}

for token in Counter(tokens).keys():
  if token not in vocab:
    vocab[token] = len(vocab)

len(vocab)

352

In [7]:
# extract sentences from data
input_sentences = document.split('.')

In [8]:
def text_to_indices(sentence, vocab):
  numerical_sentence=[]
  for token in sentence:
    if token in vocab:
      numerical_sentence.append(vocab[token])
    else:
      numerical_sentence.append(vocab['<unk>'])
  return numerical_sentence

In [9]:
#numerical sentences
input_numerical_sentences = []
for sentence in input_sentences:
  res = text_to_indices(word_tokenize(sentence.lower()), vocab)
  input_numerical_sentences.append(res)
len(input_numerical_sentences)

64

In [10]:
traing_sequence = []
for sentence in input_numerical_sentences:
  for i in range(1, len(sentence)):
    traing_sequence.append(sentence[:i+1])
# len(traing_sequence)
traing_sequence[:5]

[[1, 2], [1, 2, 3], [1, 2, 3, 4], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5, 6]]

In [11]:
#Padding
padded_training_sequence = []
len_list = []
for sequence in traing_sequence:
  len_list.append(len(sequence))
max_len = max(len_list)
print(max_len)

for sequence in traing_sequence:
  padded_training_sequence.append([0]*(max_len-len(sequence)) + sequence)
len(padded_training_sequence[0])

53


53

In [12]:
padded_training_sequence = torch.tensor(padded_training_sequence, dtype=torch.long)
padded_training_sequence.shape

torch.Size([1160, 53])

In [13]:
X = padded_training_sequence[:, :-1]
Y = padded_training_sequence[:, -1]

In [14]:
class CustomDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, index):
    return self.X[index], self.y[index]


In [15]:
dataset = CustomDataset(X, Y)
X.shape, Y.shape

(torch.Size([1160, 52]), torch.Size([1160]))

In [16]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [17]:
class LSTMModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, 100)
    self.lstm = nn.LSTM(100, 150, batch_first=True)
    self.fc = nn.Linear(150, vocab_size)

  def forward(self, x):
    embeded =self.embedding(x)
    intermediate_states, (final_hidden_state, final_cell_state) = self.lstm(embeded)
    output = self.fc(final_hidden_state.squeeze(0))
    return output


In [18]:
# example with single item ->> Main model below
x = nn.Embedding(len(vocab), 100)
y = nn.LSTM(100, 150, batch_first=True)
z = nn.Linear(150, len(vocab))
a = dataset[0][0].unsqueeze(0)
b = x(a)
c, d = y(b)
e, f =  d
f.shape
z(f.squeeze(0)).shape


torch.Size([1, 352])

In [25]:
model = LSTMModel(len(vocab))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
epochs = 50
Learning_rate = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=Learning_rate)


In [27]:
#Training Loop

for epoch in range(epochs):
  total_loss = 0;
  for x, y in dataloader:
    x.to(device), y.to(device)
    optimizer.zero_grad()
    output = model(x)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  print(f'Epoch: {epoch+1}, Loss: {total_loss/len(dataloader)}')

Epoch: 1, Loss: 5.66275287318874
Epoch: 2, Loss: 5.006022775495374
Epoch: 3, Loss: 4.620555993672964
Epoch: 4, Loss: 4.197022051424594
Epoch: 5, Loss: 3.8021554689149597
Epoch: 6, Loss: 3.4075716379526497
Epoch: 7, Loss: 3.029614016816423
Epoch: 8, Loss: 2.696822888142354
Epoch: 9, Loss: 2.3998907482301868
Epoch: 10, Loss: 2.1071439530398393
Epoch: 11, Loss: 1.8469035657676491
Epoch: 12, Loss: 1.6128028566772874
Epoch: 13, Loss: 1.413062833450936
Epoch: 14, Loss: 1.2282224684148222
Epoch: 15, Loss: 1.0652547994175472
Epoch: 16, Loss: 0.9367614404575245
Epoch: 17, Loss: 0.8147198026244705
Epoch: 18, Loss: 0.719545999894271
Epoch: 19, Loss: 0.6394835203080564
Epoch: 20, Loss: 0.5698609867611447
Epoch: 21, Loss: 0.5119010789974315
Epoch: 22, Loss: 0.4580898180201247
Epoch: 23, Loss: 0.4122038983009957
Epoch: 24, Loss: 0.3829249526197846
Epoch: 25, Loss: 0.34970812459249756
Epoch: 26, Loss: 0.32002144890862544
Epoch: 27, Loss: 0.2919019589553008
Epoch: 28, Loss: 0.2715678500968057
Epoch: 2

In [66]:
def prediction(model, vocab, sentence):
  # tokinzed_text =
  tokenized_text = word_tokenize(sentence.lower())

  # text to numerical text
  numerical_text = text_to_indices(tokenized_text, vocab)

  # padding
  padded_text = torch.tensor([0] * (max_len-len(numerical_text)) + numerical_text, dtype=torch.long).unsqueeze(0)

  # send to model
  output = model(padded_text);

  #predicted index
  value, index = torch.max(output, dim=1)

  #merge with text
  return sentence+list(vocab.keys())[index]


In [67]:
prediction(model, vocab, "Hi, I am late ")

'Hi, I am late ,'

In [73]:
import time
num_tokens = 10
input_text = "I have read that Job"
for i in range(num_tokens):
 output = prediction(model, vocab, input_text)
 print(output)
 input_text = (output+" ")
 time.sleep(1)


I have read that Jobpreparation
I have read that Jobpreparation and
I have read that Jobpreparation and i
I have read that Jobpreparation and i am
I have read that Jobpreparation and i am unable
I have read that Jobpreparation and i am unable to
I have read that Jobpreparation and i am unable to make
I have read that Jobpreparation and i am unable to make the
I have read that Jobpreparation and i am unable to make the payment
I have read that Jobpreparation and i am unable to make the payment ?
